## Домашнее задание «Продвинутый pandas 2»
### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [94]:
import pandas as pd

ratings = pd.read_csv('addition/ml-latest-small/ratings.csv')
lifecycle = ratings.groupby('userId').agg(['min', 'max', 'count'])['timestamp'].query('count > 100')
print ('User mean lifecycle time  (100+ assesment) is', lifecycle['max'].mean() - lifecycle['min'].mean())

User mean lifecycle time  (100+ assesment) is 40080507.44961238


### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.
________________________________________________________________________________
##### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [95]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
#_#
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
#_#
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
#_#
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [100]:
rzd, auto, air, client_base

(   client_id  rzd_revenue
 0        111         1093
 1        112         2810
 2        113        10283
 3        114         5774
 4        115          981,
    client_id  auto_revenue
 0        113         57483
 1        114            83
 2        115           912
 3        116          4834
 4        117            98,
    client_id  air_revenue
 0        115           81
 1        116            4
 2        117           13
 3        118          173,
    client_id          address
 0        111  Комсомольская 4
 1        112   Энтузиастов 8а
 2        113  Левобережная 1а
 3        114          Мира 14
 4        115        ЗЖБИиДК 1
 5        116    Строителей 18
 6        117  Панфиловская 33
 7        118     Мастеркова 4)

In [135]:
rzd_track = rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id')
rzd_track_detiled = rzd_track.merge(client_base, how='outer', on='client_id')
rzd_track_detiled

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.
### Ответ
Насколько я понял задача связана с посещением и/или подтвержденной покупки пользователя интернет магазина. Т.е. необходимо проанализировать платежеспособность локации (региона, района, города). И/или предрасположенность пользователей в той или иной локации к приобретению тех или иных товаров.
 Допустим помимо координат у нас есть еще один датафрейм с user_ID и статистикой его посещений и покупок. Для определения местоположения человека во время покупки как видится необходимо сопоставить время покупки и имеющиеся координаты пользователя объединив в один датафрейм методом outline merge. Из датафрейма можем извлечь всю информацию для пользователя, региона, города, время посещений, покупок, инетерсующие группы товаров, платежеспособность.
  Разница координат вряд ли повлияет на расчет критично, в случае соотношения локаций при покупке или визите для user_ID мы можем потерять точность при присваивании одной локации, что опять же таки зависит от масштаба анализа. Мы можем определить статистически наиболее вероятную локацию, к примеру если из 10 пар координат 5 или 6 идентичные, или находятся в пределах погрешности определения локации. Второй вариант когда мы для каждого уникального пользователя можем присваивать несколько локаций в подгруппе.
 Из координат локации/й пользователя можно извлечь адреса, снимки привязанные к геосервисам, погоду, состояние дорог, близлежащие магазины и т.д. из чего можно определить его ориентировочную платежеспособность и многое другое.